Install libs and import them

In [ ]:
!pip install nltk
import nltk
import subprocess
from nltk import *
from nltk.sem import Expression

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Install prover9 from external source [Currently not working]

In [ ]:
# !wget https://www.cs.unm.edu/~mccune/prover9/download/LADR-2009-11A.tar.gz
# !tar -xzvf LADR-2009-11A.tar.gz
# !cd LADR-2009-11A
# !make all

# prover = nltk.Prover9()
# prover.config_prover9(binary_location='LADR-2009-11A')


A sample First order logic validation with expression and "assumptions"

In [ ]:
read_expr = Expression.fromstring
p1 = read_expr('man(socrates)')
p2 = read_expr('all x.(man(x) -> mortal(x))')
c  = read_expr('mortal(socrates)')

print(TableauProver().prove(c, [p1,p2], verbose=True))

print(ResolutionProver().prove(c, [p1,p2], verbose=True))

man(socrates)
   -mortal(socrates)
      all x.(man(x) -> mortal(x)):   []
            --> Using 'socrates'
         (man(socrates) -> mortal(socrates))
            -man(socrates)
               CLOSED
            mortal(socrates)
               CLOSED
True
[1] {-mortal(socrates)}     A 
[2] {man(socrates)}         A 
[3] {-man(z2), mortal(z2)}  A 
[4] {-man(socrates)}        (1, 3) 
[5] {mortal(socrates)}      (2, 3) 
[6] {}                      (1, 5) 

True


Now, same expression without assumptions.

In [ ]:
read_expr = Expression.fromstring
c  = read_expr('mortal(socrates)')
print(TableauProver().prove(c, verbose=True))

-mortal(socrates)
   AGENDA EMPTY
False


From logicllama paper,

```∃x entire(x) <-> ¬serious(x)```

but this doesn't validate.

In [ ]:
from nltk.sem import Expression

read_expr = Expression.fromstring

# From logicllama paper
# ∃x entire(x) <-> ¬serious(x)
expr_1 = read_expr('exists x.(entire(x) <-> ¬serious(x))')

# Prove the first implication
proof_1 = TableauProver().prove(expr_1, verbose=True)
print(proof_1)

-exists x.(entire(x) <-> ¬serious(x))
   all x.-(entire(x) <-> ¬serious(x)):   []
         --> Using 'z3'
      -(entire(z3) <-> ¬serious(z3))
         entire(z3)
            -¬serious(z3)
               all x.-(entire(x) <-> ¬serious(x)):   [z3]
                     --> Variables Exhausted
                  AGENDA EMPTY
False


In [ ]:
from nltk.sem import Expression
from nltk.inference.tableau import TableauProver
from nltk.inference.resolution import ResolutionProver

read_expr = Expression.fromstring

# FOL statements
# ∀x (entire(x) -> ¬serious(x)) ∧ ∀x (serious(x) -> ¬entire(x))
expr_combined = read_expr('all x.(entire(x) -> ¬serious(x)) & all x.(serious(x) -> ¬entire(x))')

# Assumptions
assumptions = [
    read_expr('entire(x) -> ¬serious(x)'),  # Assumption: John is entire
    read_expr('serious(x) -> ¬entire(x)'),  # Assumption: John is serious
    # Add more assumptions if known or relevant
]

# Adding assumptions to the proof
# Using Tableau Prover
proof_tableau = TableauProver().prove(expr_combined, assumptions)
print(proof_tableau)  # This will print True if the proof is successful

# Using Resolution Prover
proof_resolution = ResolutionProver().prove(expr_combined, assumptions, verbose=True)
print(proof_resolution)  # This will show the resolution steps if the proof is successful


False
[ 1] {entire(z8), serious(z7)}         A 
[ 2] {entire(z10), -¬entire(z9)}       A 
[ 3] {-¬serious(z12), serious(z11)}    A 
[ 4] {-¬serious(z14), -¬entire(z13)}   A 
[ 5] {-entire(z15), ¬serious(z15)}     A 
[ 6] {-serious(z16), ¬entire(z16)}     A 
[ 7] {¬serious(z15), serious(z7)}      (1, 5) 
[ 8] {¬entire(z16), entire(z8)}        (1, 6) 
[ 9] {¬serious(z15), -¬entire(z9)}     (2, 5) 
[10] {-serious(z16), entire(z10)}      (2, 6) 
[11] {entire(z10), entire(z8)}         (1, 10) 
[12] {entire(z8), entire(z10)}         (2, 8) 
[13] {-entire(z15), serious(z11)}      (3, 5) 
[14] {serious(z11), serious(z7)}       (1, 13) 
[15] {serious(z11), -¬entire(z9)}      (2, 13) 
[16] {¬entire(z16), -¬serious(z12)}    (3, 6) 
[17] {-¬serious(z12), entire(z10)}     (2, 16) 
[18] {serious(z7), serious(z11)}       (3, 7) 
[19] {-¬entire(z9), serious(z11)}      (3, 9) 
[20] {entire(z10), -¬serious(z12)}     (3, 10) 
[21] {-entire(z15), -¬entire(z13)}     (4, 5) 
[22] {-¬entire(z13), serious(z7)

LogicLlama inference

In [ ]:
!pip install bitsandbytes
!pip install accelerate
!pip install peft
!pip install datasets transformers[sentencepiece]
!pip install sentencepiece

import torch
from functools import partial
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from peft import PeftModel, prepare_model_for_int8_training
# from utils import TranslationDataPreparer, ContinuousCorrectionDataPreparer, make_parent_dirs
# from fol_parser import parse_text_FOL_to_tree
# from generate import llama_generate

base_model='huggyllama/llama-7b' # TODO: fill in with the path to the llama-7b model
prompt_template_path='data/prompt_templates'
load_in_8bit = True
max_output_len = 128

tokenizer = LlamaTokenizer.from_pretrained(base_model)
tokenizer.add_special_tokens({
    "eos_token": "</s>",
    "bos_token": "<s>",
    "unk_token": '<unk>',
    "pad_token": '<unk>',
})
tokenizer.padding_side = "left"  # Allow batched inference

generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=1
)



In [ ]:
llama_model = LlamaForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map='auto',
)
llama_model = prepare_model_for_int8_training(llama_model)

peft_path='yuan-yang/LogicLLaMA-7b-direct-translate-delta-v0'
model = PeftModel.from_pretrained(
    llama_model,
    peft_path,
    torch_dtype=torch.float16
)
# model.to('cuda')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:136: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
